This notebook is for testing of:
- Data transformation from a "cleaned, preprocessed, splited" pickle file 
- Model loading
- Model training

In [4]:
PROJECT_DIR = "C:/Users/Uyen/Documents/nlp/Thesis/aspect-targeted-polarity-classification"


In [2]:
import sys
import os

sys.path.append(PROJECT_DIR)
sys.path.append(os.path.join(PROJECT_DIR, "finetuning_and_classification"))


import utils
from utils_glue import (compute_metrics, convert_examples_to_features,
                        output_modes, processors)


In [3]:
# Test data load from pkl
sentences, aspect_term_sentiments =utils.YNmilk_data_reform("C:\\Users\\Uyen\\Documents\\nlp\\YNdata\\data_splited\\train.pickle")

sentences[0:5], aspect_term_sentiments[0:5]

(["'ngày uống được 3 cữ thì tốt nhất chị ạ'",
  "'Thơm ngon bổ dưỡng'",
  "'Thơm ngon bổ dưỡng'",
  "'Minh Anh Leđừng quên ghé ngay cửa hàng Giấc mơ sữa Việt để mua ngay sữa Yoko để rinh những món quà đáng yêu cho bé nha😄'",
  "'Hay quá'"],
 [[('Generic', 'POS')],
  [('Taste', 'POS')],
  [('Nutrition', 'POS')],
  [('Gift', 'POS')],
  [('Online', 'POS')]])

In [25]:
os.chdir(PROJECT_DIR)
!python prepare_YNmilk_datasets.py \
    --files "C:\\Users\\Uyen\\Documents\\nlp\YNdata\\data_splited\\train.pickle" \
    --output_dir data/transformed/yndata \
    --istrain

data/transformed/yndata
C:\\Users\\Uyen\\Documents\\nlp\YNdata\\data_splited\\train.pickle
Dataset: Train
#Sentences with minimum 1 label 178065
Label Counts [('POS', 145849), ('NEU', 29108), ('NEG', 3108)]
#SentencePairs 178065
POS% 0.8190773032319658
NEG% 0.01745430039592284
NEU% 0.1634683963721113
POS/NEG 46.926962676962674
POS/NEU 5.010615638312491
NEG/NEU 0.10677476982272914

Dataset: Dev
#Sentences with minimum 1 label 17806
Label Counts [('POS', 14681), ('NEU', 2825), ('NEG', 300)]
#SentencePairs 17806
POS% 0.824497360440301
NEG% 0.016848253397731102
NEU% 0.15865438616196786
POS/NEG 48.93666666666667
POS/NEU 5.196814159292035
NEG/NEU 0.10619469026548672

Dataset: TrainSplit
#Sentences with minimum 1 label 160259
Label Counts [('POS', 131168), ('NEU', 26283), ('NEG', 2808)]
#SentencePairs 160259
POS% 0.8184750934424899
NEG% 0.0175216368503485
NEU% 0.16400326970716153
POS/NEG 46.712250712250714
POS/NEU 4.990602290453905
NEG/NEU 0.10683711905033672



In [24]:
!python prepare_YNmilk_datasets.py \
--files "C:\\Users\\Uyen\\Documents\\nlp\YNdata\\data_splited\\validate.pickle" \
--output_dir data/transformed/yndata 

data/transformed/yndata
C:\\Users\\Uyen\\Documents\\nlp\YNdata\\data_splited\\validate.pickle
Dataset: Test
#Sentences with minimum 1 label 38427
Label Counts [('POS', 31555), ('NEU', 6166), ('NEG', 706)]
#SentencePairs 38427
POS% 0.8211674083326828
NEG% 0.018372498503656284
NEU% 0.16046009316366097
POS/NEG 44.695467422096314
POS/NEU 5.117580278949076
NEG/NEU 0.11449886474213429



In [11]:
import xml.etree.ElementTree as ET

with open("C:/Users/Uyen/Documents/nlp/thesis/aspect-targeted-polarity-classification/data/transformed/yndata/testtest.xml", "r", encoding="utf-8") as file:
    sentence_elements = ET.parse(file).getroot().iter('sentence')

ParseError: no element found: line 8, column 13 (<string>)

In [12]:
os.chdir(PROJECT_DIR)

MODEL_DIR = os.path.join(PROJECT_DIR, "data/models/phobert_base/huggingface_phobert_base")
DATA_DIR = os.path.join(PROJECT_DIR, "data/transformed/yndata")
OUTPUT_DIR = os.path.join(PROJECT_DIR, "YNData-atsc-phobert-milk")

!python ./finetuning_and_classification/run_glue.py \
    --model_type="phobert" \
    --model_name_or_path=C:/Users/Uyen/Documents/nlp/thesis/aspect-targeted-polarity-classification/data/models/phobert_base/huggingface_phobert_base \
    --do_train \
    --evaluate_during_training \
    --logging_steps 100 --save_steps 200 --task_name="atsc" \
    --seed 42 \
    --do_lower_case \
    --data_dir="$DATA_DIR" \
    --output_dir="$OUTPUT_DIR" \
    --max_seq_length=128 \
    --learning_rate 3e-5 \
    --per_gpu_eval_batch_size=32 \
    --gradient_accumulation_steps=1 \
    --max_steps=800 \
    --overwrite_output_dir --overwrite_cache --warmup_steps=120
    

02/25/2021 20:57:38 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
Some weights of the model checkpoint at C:/Users/Uyen/Documents/nlp/thesis/aspect-targeted-polarity-classification/data/models/phobert_base/huggingface_phobert_base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model 

In [8]:
# from transformers import (WEIGHTS_NAME, BertConfig,
#                                   BertForSequenceClassification, BertTokenizer)


# config_class, model_class, tokenizer_class = (BertConfig, BertForSequenceClassification, BertTokenizer)
# config = config_class.from_pretrained(MODEL_DIR, num_labels=num_labels, finetuning_task=task_name)
# tokenizer = tokenizer_class.from_pretrained(MODEL_DIR, do_lower_case=True)
# model = model_class.from_pretrained(MODEL_DIR, from_tf=bool('.ckpt' in MODEL_DIR), config=config)
